# Play with 3dshapes

### Import

In [1]:
import os
PROJECT_PATH="/home/alexandre/disdiff_adaptaters"
os.chdir(PROJECT_PATH)

import lightning as L
from lightning import LightningDataModule, LightningModule

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import h5py

from disdiff_adaptaters.data_module.shapes3d import Shapes3DDataModule

## Load data

In [ ]:
data_module = Shapes3DDataModule()
train_loader = data_module.train_dataloader()